# 使用 GNN 进行链接预测

link prediction: 预测图中两个任意节点之间的边是否存在

In [1]:
import itertools
import os
os.environ['DGLBACKEND'] = 'pytorch'

import dgl
import dgl.data
import numpy as np
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F

## Link prediction 概述

许多应用，例如社交推荐、项目推荐、知识图等，都可以被视为链接预测，它预测两个特定节点之间是否具有边。  
本教程展示的是，预测引用网络中，两篇论文之间是否存在引用关系。  
本教程中，链接预测问题被转化为二分类问题：
 - 将图中的边视为 positive 样本
 - 对图中不存在的边采样，作为 negative 样本
 - 将阳性和阴性样本划分至训练集与测试集
 - 使用二分类指标如 AUC 评价模型

## 载入图和特征

In [2]:
dataset = dgl.data.CoraGraphDataset()
g = dataset[0]
g

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'feat': Scheme(shape=(1433,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'train_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})

## 准备训练集和测试集

本教程中，随机选取 10% 的边作为测试集，其余组成训练集；对于负样本，样本数保持一致

In [3]:
u, v = g.edges()

u

tensor([   0,    0,    0,  ..., 2707, 2707, 2707])

In [4]:
v

tensor([ 633, 1862, 2582,  ...,  598, 1473, 2706])

In [5]:
eids = np.arange(g.num_edges())
eids = np.random.permutation(eids)
eids

array([4265, 6864, 3395, ..., 5679, 5080, 6790])

In [6]:
test_size = int(len(eids) * 0.1)
train_size = g.num_edges() - test_size
print(train_size, test_size)

9501 1055


In [7]:
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

In [12]:
# Find all negative edges and split them for training and testing
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
adj

<2708x2708 sparse matrix of type '<class 'numpy.float64'>'
	with 10556 stored elements in COOrdinate format>

In [13]:
adj_neg = 1 - adj.todense() - np.eye(g.num_nodes())
adj_neg

matrix([[0., 1., 1., ..., 1., 1., 1.],
        [1., 0., 0., ..., 1., 1., 1.],
        [1., 0., 0., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 0., 1., 1.],
        [1., 1., 1., ..., 1., 0., 0.],
        [1., 1., 1., ..., 1., 0., 0.]])

In [16]:
neg_u, neg_v = np.where(adj_neg != 0)
neg_u

array([   0,    0,    0, ..., 2707, 2707, 2707])

In [17]:
neg_u.shape

(7320000,)

In [18]:
neg_eids = np.random.choice(len(neg_u), g.num_edges())
neg_eids

array([4544370, 2721055,  682720, ..., 5945396,  509805, 7203401])

In [19]:
neg_eids.shape

(10556,)

In [20]:
test_neg_u, test_neg_v = (
    neg_u[neg_eids[:test_size]],
    neg_v[neg_eids[:test_size]],
)
train_neg_u, train_neg_v = (
    neg_u[neg_eids[test_size:]],
    neg_v[neg_eids[test_size:]],
)

训练时，应该删除在测试集中的边，这可以通过`dgl.remove_edges`实现  
`dgl.remove_edges` 根据原始图创建子图

In [21]:
train_g = dgl.remove_edges(g, eids[:test_size])
train_g

Graph(num_nodes=2708, num_edges=9501,
      ndata_schemes={'feat': Scheme(shape=(1433,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'train_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})

## Define a GraphSAGE model

 - This tutorial builds a model consisting of two GraphSAGE layers, each computes new node representations by averaging neighbor information.
 - DGL provides `dgl.nn.SAGEConv` that conveniently creates a GraphSAGE layer.

In [30]:
from dgl.nn import SAGEConv

class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, "mean")
        self.conv2 = SAGEConv(h_feats, h_feats, "mean")
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

The model then predicts the probability of existence of an edge by computing a score between the representations of both incident nodes with a function.（模型通过计算两个时间节点的表示之间的分数来预测边存在的概率）

## Positive graph, negative graph, and `apply_edges`

 - link prediction requires you to compute representation of *pairs of nodes*
 - DGL recommends you to treat the pairs of nodes as another graph.
 - In link prediction, you will have a *positive graph* consisting of all the positive examples as edges, and a *negative graph* consisting of all the negative examples.
 - The *positive graph* and the *negative graph* will contain the same set of nodes as the original graph.

In [24]:
train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.num_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.num_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.num_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.num_nodes())

 - The benifit of treating the pairs of nodes as a graph is that you can use the `DGLGraph.apply_edges` method, which compute new edge features based on the incident nodes' features and the original edge features.
 - DGL provides a set of optimized builtin functions to compite new edge features based on the original node/edge features. E.g., `dgl.function.u_dot_v` 

In [25]:
import dgl.function as fn

class DotPredictor(nn.Module):
    def forward(self, g, h):
        g.ndata['h'] = h
        # compute a new edge feature named 'score' by a dot-product
        # between the source node feature 'h' and the destination node feature 'h'.
        g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
        # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
        return g.edata['score'][:, 0]

 - You can also write your own function if it is complex.
 - The following module produces a scalar score on each edge by concatenating the incident nodes' features and passing it to an MLP.

In [26]:
class MLPPredictor(nn.Module):
    pass

**Note**: The builtin functions are optimized for both speed and memory. We recommend using builtin functions whenever possible.

## Training loop

 - The loss function is simply binary cross entropy loss.
 - The evaluation metric in this tutorial is AUC.

In [31]:
model = GraphSAGE(train_g.ndata['feat'].shape[1], 16)
pred = DotPredictor()

def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]
    )
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]
    ).numpy()
    return roc_auc_score(labels, scores)

In [32]:
optimizer = torch.optim.Adam(
    itertools.chain(model.parameters(), pred.parameters()), lr=0.01
)

all_logits = []
for e in range(100):
    # forward
    h = model(train_g, train_g.ndata['feat'])
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score)
    
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if e % 5 == 0:
        print('In epoch {}, loss {:.3f}'.format(e, loss))

In epoch 0, loss 0.711
In epoch 5, loss 0.692
In epoch 10, loss 0.681
In epoch 15, loss 0.658
In epoch 20, loss 0.621
In epoch 25, loss 0.567
In epoch 30, loss 0.525
In epoch 35, loss 0.503
In epoch 40, loss 0.482
In epoch 45, loss 0.463
In epoch 50, loss 0.448
In epoch 55, loss 0.432
In epoch 60, loss 0.416
In epoch 65, loss 0.399
In epoch 70, loss 0.382
In epoch 75, loss 0.365
In epoch 80, loss 0.350
In epoch 85, loss 0.333
In epoch 90, loss 0.317
In epoch 95, loss 0.301


In [33]:
from sklearn.metrics import roc_auc_score

In [34]:
with torch.no_grad():
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    print('AUC {}'.format(compute_auc(pos_score, neg_score)))

AUC 0.8601504907796321
